# **SPARK**

## **Instalamos e importamos librerías**

In [1]:
!pip install pyspark
!pip install -u -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from google.colab import drive
drive.mount('/content/drive')
!pip install pydrive


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u
Hit:1 https://cli.github.com/packages stable InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,065 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-ba

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Creo el Spark Context

In [5]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

## Leo los CSV's

In [6]:
SQLContext = SQLContext(sc)

/usr/local/lib/python3.12/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [7]:
dfcustomers = SQLContext.read.csv("/content/drive/MyDrive/dataTP2/customers.csv", header=True, inferSchema=True)
rddcustomers = dfcustomers.rdd

In [8]:
dfCategories = SQLContext.read.csv("/content/drive/MyDrive/dataTP2/categories.csv", header=True, inferSchema=True)
rddCategories = dfCategories.rdd

In [9]:
dfinventoryLogs = SQLContext.read.csv("/content/drive/MyDrive/dataTP2/inventory_logs.csv", header=True, inferSchema=True)
rddInventoryLogs = dfinventoryLogs.rdd

In [10]:
dfOrderItems = SQLContext.read.csv("/content/drive/MyDrive/dataTP2/order_items.csv", header=True, inferSchema=True)
rddOrderItems = dfOrderItems.rdd

In [11]:
dfOrders = SQLContext.read.csv("/content/drive/MyDrive/dataTP2/orders.csv", header=True, inferSchema=True)
rddOrders = dfOrders.rdd

In [12]:
dfProducts = SQLContext.read.csv("/content/drive/MyDrive/dataTP2/products.csv", header=True, inferSchema=True)
rddProducts = dfProducts.rdd

In [13]:
dfReviews = SQLContext.read.csv("/content/drive/MyDrive/dataTP2/reviews.csv", header=True, inferSchema=True)
rddReviews = dfReviews.rdd

# **LIMPIEZA DE DATOS**

In [14]:
INCORRECTO = {"undefined","null","none",""}

def to_float(x):
    try: return float(x)
    except: return None

def extract_state(addr: str):
    if not addr: return None
    final_cadena = str(addr).strip().rsplit(",", 1)[-1].strip()      # "AZ 30649"
    first = final_cadena.split()[0] if final_cadena else ""
    return first.upper() if len(first)==2 and first.isalpha() else None

def extraer_codigo_postal(addr:str):
    if not addr: return None
    final_cadena = str(addr).strip().split()[-1].lower()
    if final_cadena not in INCORRECTO:
      first = final_cadena.split()[0]
      return first
    else:
      return None


def normalizar_string(s):
    if es_string_correcto(s):
      s = s.title().strip()
      return s
    else :
      return None

def es_string_correcto(s):
    if s is not None and s.strip().lower() not in INCORRECTO:
      return True
    else:
      return False

# **CONSULTAS**



1) Cuál es el estado que más descuentos tiene en total? y en promedio? Supongan que de una direccion del estilo: 3123 Alan Extension Port Andrea, MA 26926, “MA” es el estado.

2) ¿Cuáles son los 5 códigos postales más comunes para las órdenes con estado ‘Refunded’? ¿Y cuál es el nombre más frecuente entre los clientes de esas direcciones?

3) Para cada tipo de pago y segmento de cliente, devolver la suma y el promedio expresado como porcentaje, de clientes activos y de consentimiento de marketing. Se valora que el output de la consulta tenga nombres claros y en español.

4) Para los productos que contienen en su descripción la palabra “stuff” (sin importar mayúsculas o minúsculas), calcular el peso total de su inventario agrupado por marca, mostrar sólo la marca y el peso total de las 5 más pesadas.

5) Calculen el porcentaje de productos cuyo stock es al menos 20% más alto que el stock promedio de su marca. Por ejemplo, si el stock promedio de la marca Adidas fuera 100, para los productos de dicha marca la condición será que tengan un stock mayor a 120, y luego se deberá calcular qué porcentaje del total de productos cumple con esta condición.

6) Obtener la cantidad de órdenes que no hayan comprado ninguno de los 10 productos más vendidos.

## 1. Cuál es el estado que más descuentos tiene en total? y en promedio? Supongan que de una direccion del estilo: 3123 Alan Extension Port Andrea, MA 26926, “MA” es el estado

In [ ]:
rddOrders.take(1)

[Row(_c0=0, order_id=1, customer_id=447917, order_date='2024-07-26T03:04:05.462241', status='completed', payment_method='Digital Wallet', shipping_address='49599 Wesley Burg Richardview, AZ 30649', billing_address='USNV Morrison FPO AP 90901', discount_amount=0.0, tax_amount=None, shipping_cost=None, total_amount=25.96, currency='USD', created_at='2024-07-26T03:04:05.462241', updated_at='2024-07-27T03:04:05.462241', subtotal=0)]

In [ ]:
#discount_amount, shipp_addres

In [ ]:

rddEstadosConDescuentos = rddOrders.filter(lambda r: r.shipping_address and r.discount_amount is not None).map(lambda r: (extract_state(r.shipping_address), to_float(r.discount_amount)))\
                              .filter(lambda kv: kv[0] is not None and kv[1] is not None and kv[1] > 0).mapValues(lambda v: (v, 1)).reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])).cache()

In [ ]:
rddEstadosConDescuentos.take(5)

[('OR', (685793.6299999999, 13657)),
 ('MS', (689506.7100000001, 13709)),
 ('LA', (684889.1499999996, 13684)),
 ('IN', (681833.2300000001, 13719)),
 ('AL', (680092.72, 13672))]

In [ ]:
# Estado con MÁS DESCUENTO TOTAL ($)
rddEstadosConDescuentos.mapValues(lambda sc: sc[0]).reduce(lambda a,b: a if a[1] > b[1] else b)

('HI', 699381.1599999999)

In [ ]:
# Estado con MÁS DESCUENTO TOTAL ($)
estado_max_total = rddEstadosConDescuentos.mapValues(lambda sc: sc[0]).reduce(lambda a,b: a if a[1] > b[1] else b)

# Estado con MAYOR PROMEDIO de descuento ($ por registro con descuento)
estado_max_prom = rddEstadosConDescuentos.mapValues(lambda sc: sc[0] / sc[1]).reduce(lambda a,b: a if a[1] > b[1] else b)

print(f"estadoConMasDescuentosTotales: {estado_max_total[0]}, ${estado_max_total[1]:,.2f}")
print(f"estadoConMasDescuentosEnPromedio: {estado_max_prom[0]}, ${estado_max_prom[1]:,.2f}")


estadoConMasDescuentosTotales: HI, $699,381.16
estadoConMasDescuentosEnPromedio: OH, $50.40


## 2. ¿Cuáles son los 5 códigos postales más comunes para las órdenes con estado ‘Refunded’? ¿Y cuál es el nombre más frecuente entre los clientes de esas direcciones?

In [ ]:
rddOrders.take(1)

[Row(_c0=0, order_id=1, customer_id=447917, order_date='2024-07-26T03:04:05.462241', status='completed', payment_method='Digital Wallet', shipping_address='49599 Wesley Burg Richardview, AZ 30649', billing_address='USNV Morrison FPO AP 90901', discount_amount=0.0, tax_amount=None, shipping_cost=None, total_amount=25.96, currency='USD', created_at='2024-07-26T03:04:05.462241', updated_at='2024-07-27T03:04:05.462241', subtotal=0)]

In [ ]:
rddcustomers.take(1)

[Row(_c0=0, customer_id=1, email='uthomas@example.net', first_name='Kayla', last_name='Smith', phone='329-672-0449x89707', date_of_birth='1948-05-20', gender='F', country='Brazil', city='south michelle', postal_code=70351, address=None, registration_date='2022-09-22T09:40:47.913063', last_login='2025-01-07T12:09:24.323425', is_active=True, customer_segment='Regular', marketing_consent=True)]

In [ ]:
rddOrdersRefundedZipCus = rddOrders.filter(lambda x: normalizar_string(x.status) == 'Refunded' and x.shipping_address is not None).map(lambda x: (extraer_codigo_postal(x.shipping_address), x.customer_id))\
    .filter(lambda kv: kv[0] is not None)


In [ ]:
rddTop5CodigoPostales = rddOrdersRefundedZipCus.map(lambda x: (x[0],1)).reduceByKey(lambda a,b: a+b).cache()

In [ ]:
# top 5 codigos postales
top5 = rddTop5CodigoPostales.takeOrdered(5, key = lambda x: -x[1])

In [ ]:
top5Zips = [z for z, _ in top5]
btop5 = sc.broadcast(set(top5Zips))

In [ ]:
ordersRefundedTop5Bycust = rddOrdersRefundedZipCus.filter(lambda zc: zc[0] in btop5.value).map(lambda zc: (zc[1], zc[0]))

In [ ]:
ordersRefundedTop5Bycust.take(1)

[(471085, '59883')]

In [ ]:
rddbyCus = rddcustomers.map(lambda clientes: (clientes.customer_id,normalizar_string(clientes.first_name))).filter(lambda x: x[1]is not None)

In [ ]:
joined  = ordersRefundedTop5Bycust.join(rddbyCus)

In [ ]:
nameCountByZip = joined.map(lambda x: ((x[1][0], x[1][1]), 1)).reduceByKey(lambda a,b: a[1]+b[1])

In [ ]:
# Top nombre por ZIP (con desempate alfabético)
topNamePerZip = nameCountByZip.map(lambda znc: (znc[0][0], (znc[0][1], znc[1]))).reduceByKey(lambda a, b: a if (a[1] > b[1]) or (a[1] == b[1] and a[0] <= b[0]) else b)


# ----------------- Resultados -----------------
result = top5                                   # [(zip, count)]
top_names_result = topNamePerZip.sortByKey().collect()# [(zip, (nombre, count))]

print("Top-5 ZIPs (Refunded):", result)
print("Nombre más frecuente por ZIP del top-5:", top_names_result)

Top-5 ZIPs (Refunded): [('70696', 6), ('47612', 5), ('11954', 5), ('83755', 5), ('59883', 5)]
Nombre más frecuente por ZIP del top-5: [('11954', ('Anthony', 1)), ('47612', ('Debbie', 1)), ('59883', ('Carolyn', 1)), ('70696', ('Amber', 1)), ('83755', ('Alexis', 1))]


## 3. Para cada tipo de pago y segmento de cliente, devolver la suma y el promedio expresado como porcentaje, de clientes activos y de consentimiento de marketing. Se valora que el output de la consulta tenga nombres claros y en español.

In [ ]:
rddOrders.take(1)

[Row(_c0=0, order_id=1, customer_id=447917, order_date='2024-07-26T03:04:05.462241', status='completed', payment_method='Digital Wallet', shipping_address='49599 Wesley Burg Richardview, AZ 30649', billing_address='USNV Morrison FPO AP 90901', discount_amount=0.0, tax_amount=None, shipping_cost=None, total_amount=25.96, currency='USD', created_at='2024-07-26T03:04:05.462241', updated_at='2024-07-27T03:04:05.462241', subtotal=0)]

In [16]:
customersMethod = rddOrders.filter(lambda x: x.customer_id is not None and x.payment_method is not None).map(lambda x: ((x.customer_id, normalizar_string(x.payment_method)),1)).filter(lambda x :x[0][1] is not None)\
                            .reduceByKey(lambda a,b: 1).map(lambda kv: (kv[0][0], kv[0][1])).cache()






In [ ]:
for r in customersMethod.take(5):
    print(r)      # (customer_id, metodo)

(420585, 'Paypal')
(467719, 'Digital Wallet')
(492286, 'Cash On Delivery')
(489351, 'Credit Card')
(438792, 'Debit Card')


In [ ]:
rddcustomers.take(1)

[Row(_c0=0, customer_id=1, email='uthomas@example.net', first_name='Kayla', last_name='Smith', phone='329-672-0449x89707', date_of_birth='1948-05-20', gender='F', country='Brazil', city='south michelle', postal_code=70351, address=None, registration_date='2022-09-22T09:40:47.913063', last_login='2025-01-07T12:09:24.323425', is_active=True, customer_segment='Regular', marketing_consent=True)]

In [17]:
#'customer_id','customer_segment','is_active','marketing_consent'
#Aca filtro aquellos registros de clientes que no tienen una fecha de registro porque intuyo que datos erroneos.
CustomersPerId = rddcustomers.filter(lambda x:  x.customer_segment is not None
                                                and x.registration_date is not None
                                                and x.is_active is not None
                                                and x.marketing_consent is not None)\
                                        .map(lambda cliente: (cliente.customer_id, (normalizar_string(cliente.customer_segment) ,cliente.is_active, cliente.marketing_consent)))\
                                        .filter(lambda x :x[1][0] is not None )\
                                        .reduceByKey(lambda a,b: b)


In [18]:
NP = 200
customersMethod = customersMethod.partitionBy(NP).cache()
CustomersSegmentMarketing = CustomersPerId.partitionBy(NP).cache()

joined = customersMethod.join(CustomersSegmentMarketing)

In [19]:
MethodAndSegment = joined.map(lambda kv: ((kv[1][0], kv[1][1][0]),                   # clave = (metodo, segmento)
                                            (1 if kv[1][1][1] else 0,                   # suma activos
                                            1 if kv[1][1][2] else 0,                    #suma marketing
                                            1))).reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1], a[2]+b[2]))


In [30]:
Porcents = MethodAndSegment.mapValues(lambda t: (
    t[2],                       # total_clientes
    100.0 * t[0] / t[2],        # % clientes_activos
    100.0 * t[1] / t[2]         # % marketing
  )
)

In [40]:
tablaFinal = Porcents.map(lambda x: (
      x[0][1],  # segmento_cliente
      x[0][0],  # metodo_pago
      x[1][0],  # total_clientes
      x[1][1],  # % clientes_activos
      x[1][2]   # % marketing
    )).sortBy(lambda x: (x[0], x[1])).cache()  # ordena por segmento, luego método de pago


print("segmento_cliente\tmetodo_pago\ttotal_clientes\t% clientes_activos\t% marketing")
for seg, met, tot, pctA, pctM in tablaFinal.collect():
    print(f"{seg}      \t{met}          \t{tot}     \t{pctA:.6f}    \t{pctM:.6f}")


segmento_cliente	metodo_pago	total_clientes	% clientes_activos	% marketing
Budget      	Bank Transfer          	16737     	89.795065    	69.671984
Budget      	Cash On Delivery          	16734     	89.805187    	69.672523
Budget      	Credit Card          	16737     	89.795065    	69.660035
Budget      	Debit Card          	16729     	89.790185    	69.639548
Budget      	Digital Wallet          	16733     	89.804578    	69.652782
Budget      	Paypal          	16733     	89.804578    	69.658758
Premium      	Bank Transfer          	17246     	89.852719    	69.842282
Premium      	Cash On Delivery          	17241     	89.849777    	69.850937
Premium      	Credit Card          	17240     	89.843387    	69.860789
Premium      	Debit Card          	17245     	89.857930    	69.852131
Premium      	Digital Wallet          	17243     	89.856753    	69.848634
Premium      	Paypal          	17240     	89.849188    	69.860789
Regular      	Bank Transfer          	51300     	89.957115    	70.13255

## 4. Para los productos que contienen en su descripción la palabra “stuff” (sin importar mayúsculas o minúsculas), calcular el peso total de su inventario agrupado por marca, mostrar sólo la marca y el peso total de las 5 más pesadas.

In [ ]:
rddProducts.take(1)

[Row(_c0='0', product_id='1', product_name='Centralized attitude-oriented synergy ', category_id=164, brand='undefined', price=120.22, cost=49.48, stock_quantity=754, weight_kg=None, dimensions='194x144x13 ', description='Hotel quickly amount care meet. Likely consider agreement citizen organization.', is_active=True, created_at=' 2024-05-12T00:44:32.354038')]

In [ ]:
# necesito brand,descripcion, stockquantity y weight_kg

In [ ]:


brandStuff = rddProducts.filter(lambda product: (product.description is not None and product.weight_kg is not None and product.stock_quantity is not None)  and ('stuff' in product.description.lower()) and (product.brand is not None))\
                            .map(lambda product: (normalizar_string(product.brand), (product.weight_kg * product.stock_quantity))).filter(lambda kv: kv[0] is not None)


In [ ]:
brandStuffByWeight = brandStuff.reduceByKey(lambda a,b: a+b)

In [ ]:
brandStuffByWeight.take(1)

[('La-Z-Boy', 692988.8300000001)]

In [ ]:
top5Weight = brandStuffByWeight.takeOrdered(5, key = lambda x: -x[1])

In [ ]:
top5Weight

[('3M', 1851279.46),
 ('Adidas', 1578522.37),
 ('Hasbro', 1506279.13),
 ('Wayfair', 1448357.5499999998),
 ('Nike', 1385209.1700000002)]

In [ ]:
# ----------------- Resultados -----------------
for i, (brand, weight) in enumerate(top5Weight, start=1):
    print(f"{i}. {brand}: {weight:,.2f} kg.")

1. 3M: 1,851,279.46 kg.
2. Adidas: 1,578,522.37 kg.
3. Hasbro: 1,506,279.13 kg.
4. Wayfair: 1,448,357.55 kg.
5. Nike: 1,385,209.17 kg.


## 5. Calculen el porcentaje de productos cuyo stock es al menos 20% más alto que el stock promedio de su marca. Por ejemplo, si el stock promedio de la marca Adidas fuera 100, para los productos de dicha marca la condición será que tengan un stock mayor a 120, y luego se deberá calcular qué porcentaje del total de productos cumple con esta condición.

In [ ]:
rddProducts.take(1)

[Row(_c0='0', product_id='1', product_name='Centralized attitude-oriented synergy ', category_id=164, brand='undefined', price=120.22, cost=49.48, stock_quantity=754, weight_kg=None, dimensions='194x144x13 ', description='Hotel quickly amount care meet. Likely consider agreement citizen organization.', is_active=True, created_at=' 2024-05-12T00:44:32.354038')]

In [ ]:
#productos válidos
productos = rddProducts.filter(lambda product: (product.stock_quantity is not None and product.stock_quantity > 0) and product.brand is not None)\
                            .map(lambda product: (normalizar_string(product.brand), product.stock_quantity)).filter(lambda kv: kv[0] is not None).map(lambda x: (x[0],x[1])).cache()

In [ ]:
productos.take(100)

In [ ]:
productos.mapValues(lambda x: (x,1)).take(1)

[('Adidas', (27, 1))]

In [ ]:
sumatoria = productos.mapValues(lambda x: (x, 1)).reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))

In [ ]:
promedios = sumatoria.mapValues(lambda s : s[0]/s[1]) # (marca, avg_stock)

In [ ]:
promedios.take(100)

[('Dr. Martens', 503.53387096774196),
 ('Marvel', 506.3117819930537),
 ('Leuchtturm1917', 503.7713573532744),
 ('Cartier', 501.2490053826352),
 ('La-Z-Boy', 495.44209322276237),
 ('Fluke', 498.1172144334636),
 ('Whole Foods', 504.2859138533178),
 ('Ikea', 497.930654507457),
 ('Sony', 500.90724485785404),
 ('Rhodia', 504.9102594064034),
 ('Scholastic', 505.8112661226639),
 ('Ghirardelli', 496.50069605568444),
 ('American Tourister', 496.3918683446273),
 ('Vetriscience', 496.6190202831994),
 ('Coca-Cola', 494.9702380952381),
 ('Faber-Castell', 499.54011321491316),
 ('Graco', 506.5003466605038),
 ('Carter’S', 502.6960252935863),
 ('Brother', 504.3475692866879),
 ("L'Oréal", 504.05333022822543),
 ('Penguin', 502.27327491785326),
 ('Lego', 499.57158836689035),
 ('Fender', 500.97626224566693),
 ('Wayfair', 504.22302491432805),
 ('Ge', 501.08093402856497),
 ('Electronic Arts', 501.0449074074074),
 ('Greenies', 505.07560070326235),
 ('The North Face', 490.50982226379796),
 ('Kraft', 509.256049

In [ ]:
# # 3) Uno cada producto con el promedio de su marca
# # productos: (marca, stock)  promedios: (marca, avg)

joined = productos.join(promedios)

In [ ]:
joined.take(1)

[('Dr. Martens', (777, 503.53387096774196))]

In [ ]:
marcados = joined.map(lambda x: 1 if x[1][0] > 1.2 * x[1][1] else 0)

In [ ]:
cumplen = marcados.sum()
total   = productos.count()         # total de productos válidos
porcentaje_global = 100.0 * cumplen / total if total else 0.0

print(f"Porcentaje global de productos con stock > 120% del promedio de su marca: {porcentaje_global:.4f}%")


Porcentaje global de productos con stock > 120% del promedio de su marca: 39.9650%


## 6. Obtener la cantidad de órdenes que no hayan comprado ninguno de los 10 productos más vendidos.

In [ ]:
rddOrderItems.take(1)

[Row(_c0=0, order_item_id=1, order_id=19900399, product_id=940377, quantity=6, unit_price=569.28, line_total=3415.68, discount_amount=0.0)]

In [ ]:
ProductsSellers = rddOrderItems.filter(lambda x: x.product_id is not None and x.order_id is not None).map(lambda x: (x.product_id,1)).reduceByKey(lambda a,b : a + b)
#orderid, product_id

In [ ]:
top10ProductsBestSellers = ProductsSellers.takeOrdered(10, key = lambda x: -x[1])

In [ ]:
top10_ids = set(pid for (pid, _) in top10ProductsBestSellers)
btop10 = sc.broadcast(top10_ids)

In [ ]:
# Para cada ítem de orden, marco pertenece al top10
#    (order_id -> 1 si contiene un producto del top10, 0 si no)
orderHasTop10 = rddOrderItems.filter(lambda x: x.product_id is not None and x.order_id is not None).map(lambda x: (x.order_id, 1 if x.product_id in btop10.value else 0))\
                             .reduceByKey(lambda a, b: a + b)  # suma de flags por orden


In [ ]:
#Órdenes que NO compraron ninguno del top10: suma == 0
ordersNotInTop10 = orderHasTop10.filter(lambda kv: kv[1] == 0).keys()

# Cantidad pedida
count_orders_not_in_top10 = ordersNotInTop10.count()
# ----------------- Resultados -----------------
print("Cantidad de órdenes que no compraron ninguno de los 10 productos más vendidos:", count_orders_not_in_top10)


Cantidad de órdenes que no compraron ninguno de los 10 productos más vendidos: 99490


# **CONSULTAS EXTRAS**

## 7. Top-3 métodos de pago por cantidad de órdenes completadas en 2024

In [ ]:
def extraer_anio_iso(s):
    if not s: return None
    s = str(s).strip()
    return int(s[:4]) if len(s) >= 4 and s[:4].isdigit() else None

metodosDePagoCompletadas2024 = rddOrders.filter(lambda order: order.payment_method is not None and order.created_at is not None and order.status is not None)\
          .map(lambda x:(normalizar_string(x.payment_method), (normalizar_string(x.status), extraer_anio_iso(x.order_date))))\
          .filter(lambda t: t[0] is not None and t[1][0] == 'Completed' and t[1][1] == 2024).map(lambda x: (x[0],1)).reduceByKey(lambda a,b: a+b)


In [ ]:
metodosDePagoCompletadas2024.takeOrdered(3,lambda x: -x[1])

[('Digital Wallet', 143827), ('Bank Transfer', 143186), ('Paypal', 143167)]

## 8. Precio promedio por categoría padre



RDDs: rddProducts, rddCategories


In [ ]:
rddProducts.take(1)

[Row(_c0='0', product_id='1', product_name='Centralized attitude-oriented synergy ', category_id=164, brand='undefined', price=120.22, cost=49.48, stock_quantity=754, weight_kg=None, dimensions='194x144x13 ', description='Hotel quickly amount care meet. Likely consider agreement citizen organization.', is_active=True, created_at=' 2024-05-12T00:44:32.354038')]

In [ ]:
catPrice = rddProducts.filter(lambda prod: prod.price and prod.category_id is not None and prod.price > 0).map(lambda x: (x.category_id, x.price))

In [ ]:
catPrice.take(100)

[(164, 120.22),
 (9, 19.75),
 (57, 454.72),
 (56, 957.64),
 (118, 540.87),
 (12, 67.14),
 (42, 752.96),
 (63, 171.77),
 (128, 401.68),
 (175, 427.94),
 (175, 885.94),
 (68, 972.17),
 (79, 247.06),
 (33, 133.65),
 (52, 715.82),
 (151, 558.38),
 (19, 519.01),
 (122, 809.42),
 (168, 984.32),
 (115, 148.78),
 (139, 837.67),
 (100, 272.55),
 (49, 15.54),
 (136, 165.86),
 (53, 673.03),
 (19, 19.22),
 (95, 292.14),
 (142, 306.38),
 (54, 720.44),
 (3, 344.5),
 (11, 449.97),
 (61, 865.96),
 (69, 167.61),
 (59, 230.69),
 (174, 981.34),
 (68, 47.87),
 (148, 198.18),
 (172, 205.06),
 (77, 512.05),
 (174, 750.65),
 (149, 610.59),
 (73, 520.3),
 (12, 26.75),
 (99, 499.43),
 (179, 522.78),
 (144, 599.5),
 (122, 455.57),
 (55, 73.59),
 (179, 29.34),
 (140, 801.87),
 (104, 726.94),
 (98, 285.47),
 (18, 783.42),
 (30, 156.67),
 (51, 4.72),
 (89, 537.42),
 (138, 776.9),
 (126, 559.62),
 (178, 250.87),
 (50, 11.6),
 (76, 235.21),
 (33, 129.38),
 (14, 27.49),
 (39, 122.62),
 (58, 777.94),
 (157, 960.51),
 

In [ ]:
rddCategories.take(1)

[Row(_c0=0, category_id=1, category_name='Smartphones', parent_category='ELECTRONICS', created_at=None)]

In [ ]:
categories = rddCategories.filter(lambda cat:cat.category_id and cat.parent_category and cat.created_at is not None).map(lambda cat: (cat.category_id, normalizar_string(cat.parent_category))).filter(lambda x:x[1] is not None)

In [ ]:
categories.take(100)

[(2, 'Electronics'),
 (5, 'Electronics'),
 (6, 'Electronics'),
 (7, 'Clothing'),
 (8, 'Clothing'),
 (9, 'Clothing'),
 (11, 'Clothing'),
 (12, 'Books'),
 (14, 'Books'),
 (15, 'Books'),
 (16, 'Books'),
 (17, 'Books'),
 (19, 'Home & Garden'),
 (20, 'Home & Garden'),
 (21, 'Home & Garden'),
 (22, 'Home & Garden'),
 (23, 'Home & Garden'),
 (24, 'Sports & Outdoors'),
 (25, 'Sports & Outdoors'),
 (26, 'Sports & Outdoors'),
 (27, 'Sports & Outdoors'),
 (28, 'Sports & Outdoors'),
 (30, 'Health & Beauty'),
 (31, 'Health & Beauty'),
 (32, 'Health & Beauty'),
 (34, 'Health & Beauty'),
 (35, 'Health & Beauty'),
 (37, 'Toys & Games'),
 (40, 'Toys & Games'),
 (41, 'Automotive'),
 (42, 'Automotive'),
 (43, 'Automotive'),
 (44, 'Automotive'),
 (45, 'Automotive'),
 (48, 'Food & Beverages'),
 (49, 'Food & Beverages'),
 (50, 'Food & Beverages'),
 (51, 'Food & Beverages'),
 (52, 'Jewelry'),
 (53, 'Jewelry'),
 (54, 'Jewelry'),
 (55, 'Jewelry'),
 (56, 'Jewelry'),
 (57, 'Shoes'),
 (58, 'Shoes'),
 (59, 'Shoes'

In [ ]:
joinCategoriesPrice = categories.join(catPrice)

In [ ]:
joinCategoriesPrice.take(1)

[(6, ('Electronics', 2616.66))]

In [ ]:
joinCategoriesPrice.map(lambda p: (p[1][0],(p[1][1],1))).take(5)

[('Electronics', (2616.66, 1)),
 ('Electronics', (1300.89, 1)),
 ('Electronics', (198.27, 1)),
 ('Electronics', (1197.86, 1)),
 ('Electronics', (2195.56, 1))]

In [ ]:
joinCategoriesPrice.map(lambda p: (p[1][0],(p[1][1],1))).reduceByKey(lambda a,b: (a[0]+b[0], a[1] +b[1])).take(10)

[('Jewelry', (11991780.030000012, 23862)),
 ('Pet Supplies', (7112347.739999991, 14173)),
 ('Art & Crafts', (11864377.669999983, 23578)),
 ('Kitchen & Dining', (14309435.959999973, 28314)),
 ('Furniture', (16779771.499999985, 33429)),
 ('Clothing', (4781269.609999998, 18870)),
 ('Books', (1236077.600000001, 23896)),
 ('Automotive', (24148105.83000002, 23833)),
 ('Food & Beverages', (482085.8400000009, 18970)),
 ('Shoes', (11992688.229999967, 23824))]

In [ ]:
sumas_conteos = joinCategoriesPrice.map(lambda p: (p[1][0],(p[1][1],1))).reduceByKey(lambda a,b: (a[0]+b[0], a[1] +b[1]))

In [ ]:
sumas_conteos.take(1)

[('Jewelry', (11991780.030000012, 23862))]

In [ ]:
promedio_por_parent = sumas_conteos.mapValues(lambda sc: sc[0] / sc[1])

In [ ]:
promedio_por_parent.take(1)

[('Jewelry', 502.5471473472472)]

In [ ]:
top5Parent = promedio_por_parent.takeOrdered(5, key=lambda kv: -kv[1])

In [ ]:
for cat, avg in top5Parent:
    print(f"{cat}\t{avg:.2f}")

Electronics	1504.47
Automotive	1013.22
Handmade	506.24
Grocery & Gourmet Food	506.12
Industrial & Scientific	505.82
